# GPT1 - Improving Language Understanding by Generative Pre-Training

## Resumen

 * Entrenan el modelo en un gran corpus de texto sin supervisión. Modelado del lenguaje. Aprender un modelo de lenguaje de alta capacidad en un gran corpus de texto
 * Luego hacen fine-tuning en tareas de NLP supervisadas. Es decir con datasets etiquetados. Ajuste fino en una tarea objetivo con supervisión. Ajuste fino, en la que adaptamos el modelo a una tarea discriminativa con datos etiquetados. CUando evaluan al modelo en la tarea supervisada, no solo le evaluan por esa tarea, sino por lo bien que predice el siguiente token, esto ayuda a mejorar la generalización del modelo supervisado y hace que el modelo converja más rápido.
 * Utilizan la arquitectura transformer, que mejora al uso de RNN en que lo aprendido en el primer entrenamiento es más fácil de transferir a tareas supervisadas.
 * Evaluan el modelo en cuatro tipos de tareas de comprensión del lenguaje:
    * Inferencia del lenguaje natural
    * Respuesta a preguntas
    * Similitud semántica
    * Clasificación de textos. 
 * El modelo, agnóstico de tareas generales, supera a los modelos entrenados discriminativamente que emplean arquitecturas diseñadas específicamente para cada tarea, mejorando significativamente el estado del arte en 9 de las 12 tareas estudiadas.
 * También analizamos los comportamientos de "disparo cero" del modelo preentrenado en cuatro entornos diferentes y demostramos que adquiere un conocimiento lingüístico útil para las tareas posteriores.
 * En los últimos años, los investigadores han demostrado los beneficios de utilizar incrustaciones de palabras [11, 39, 42], que se entrenan en corpus no etiquetados, para mejorar el rendimiento en una variedad de tareas [8, 11, 26, 45]. Sin embargo, estos enfoques transfieren principalmente información a nivel de palabra, mientras que nosotros pretendemos capturar la semántica de nivel superior.
 * Utilizamos un decodificador Transformer multicapa
 * 